In [ ]:
import pandas as pd
import numpy as np

serie = pd.read_csv('Hotel reviews.csv')
df = pd.DataFrame(serie)

### hola

In [ ]:
# Keeping only the hotels for our business purpose
df = df[df['Establishment Type'] == 'Hotel']
df.head()

#### DATE cleaning

In [ ]:
df = df[~df['Review Date'].str.contains("ago")]
df = df[~df['Review Date'].str.contains("NEW")]
df = df.reset_index(drop = True)

In [ ]:
df

In [ ]:
df['Review Date'] = pd.to_datetime(df['Review Date'])

In [ ]:
df.dtypes

#### LOCATION cleaning

In [ ]:
df.Location[df.Location.isnull() == True] = 'NULL' #Assigning a NULL to the empty cell, without doing it it is difficult to work on this column of lists

In [ ]:
df.Location = df.Location.str.split(", ")

In [ ]:
# Creating a column just with the city and not the country because we have a lot of missing values for the country so is relevant to save only the city
for i in range(0,len(df)):
        df.Location[i] = df.Location[i][0]
        

In [ ]:
df.Location

#### Rewieer rank CLEANING

In [ ]:
df['Reviewer Rank'].fillna('Contributor', inplace=True)
df['Reviewer Rank'] = df['Reviewer Rank'].map({'Contributor': 1,'Reviewer': 2,\
                                               'Senior Contributor': 3,'Senior Reviewer': 4, 'Top Contributor': 5})

In [ ]:
df

#### Expanding the Score Breakdown into 6 different columns

In [ ]:
## creating a function to use in the below for loop
def sorter(text):
    text = text.strip()
    if text == "Value":
        return 0
    elif text == "Location":
        return 1
    elif text == "Sleep Quality":
        return 2
    elif text == "Rooms":
        return 3
    elif text == "Cleanliness":
        return 4
    elif text == "Service":
        return 5
    elif text == "Food":
        return 6
    else:
        return 7
    

In [ ]:
#Dropping Na
df = df[df['Score Breakdown'].notna()].reset_index(drop = True)

In [ ]:
import numpy as np
ma = np.zeros(shape = (len(df['Score Breakdown']), 8))

for i in range(0,len(df['Score Breakdown'])):
    
    ar = df['Score Breakdown'][i].split(";")
    ln = np.zeros(8)
    for j in range(0,len(ar)):
        k = ar[j][0:ar[j].index(':')]
        value = ar[j][(ar[j].index(':')+1):(ar[j].index('of'))].strip()
        ln[sorter(k)] = value
    ma[i] = ln
            

In [ ]:
df['Value'] = pd.Series(np.transpose(ma)[0]).astype(int)
df['Location'] = pd.Series(np.transpose(ma)[1]).astype(int)
df['Sleep Quality'] = pd.Series(np.transpose(ma)[2]).astype(int)
df['Rooms'] = pd.Series(np.transpose(ma)[3]).astype(int)
df['Cleanliness'] = pd.Series(np.transpose(ma)[4]).astype(int)
df['Service'] = pd.Series(np.transpose(ma)[5]).astype(int)
df['Food'] = pd.Series(np.transpose(ma)[6]).astype(int)
del df['Score Breakdown']

### ML Pipeline

#### NLP Cleaning

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) # Increase cell width
display(HTML("<style>.rendered_html { font-size: 16px; }</style>")) # Increase font size

# Matplotlib conf
import matplotlib.pyplot as plt
from matplotlib import interactive
%matplotlib inline

# Seaborn conf
import seaborn as sns
sns.set_palette(sns.color_palette("seismic"))

# Needed Libraries
import sys
import pandas
import pandas as pd
import numpy as np
import operator
import string
import re

import nltk
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.stem import *
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer



from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score, GridSearchCV, ShuffleSplit


In [ ]:
import nltk
nltk.download('stopwords')
import nltk
nltk.download('wordnet')
set(stopwords.words('english'))

In [ ]:
def process_text(raw_text):
    letters_only = re.sub("[^a-zA-Z]", " ",raw_text) 
    words = letters_only.lower().split()
    
    stops = set(stopwords.words("english")) 
    not_stop_words = [w for w in words if not w in stops]
    
    wordnet_lemmatizer = WordNetLemmatizer()
    lemmatized = [wordnet_lemmatizer.lemmatize(word) for word in not_stop_words]
    
    stemmer = PorterStemmer()
    stemmed = [stemmer.stem(word) for word in lemmatized]
    
    return( " ".join( stemmed ))  

In [ ]:
# Cleaning the Overview
df['clean_overview'] = df['Overview'].apply(lambda x: process_text(x))
df.head()

In [ ]:
# Cleaning the Review
# Cleaning the Overview
df['clean_review'] = df['Review'].apply(lambda x: process_text(x))
df.head()

### Target definition
In order to define the target variable to train our NLP model we are going to use the overall rating

In [ ]:
df['Target'] = np.nan
for i in range(0,len(df)):
    if df['Star Rating'][i] > 3:
        df['Target'][i] = 1
    else:
        df['Target'][i] = 0
        

In [ ]:
df[df.Target == 0]